In [1]:
# TÍTULO: Entrenamiento del Modelo de Detección de Autos (YOLOv8)
# OBJETIVO: Entrenar una red neuronal convolucional utilizando Transfer Learning.

import torch
from ultralytics import YOLO
import os
import matplotlib.pyplot as plt
import cv2
import glob
import yaml

# --- VERIFICACIÓN DE GPU (CRÍTICO PARA TU RTX 4070 Ti) ---
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print(f"Hardware detectado: {device_name}")
    print(f"   Memoria VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    device = 0 # ID de la GPU
else:
    print("No se detectó GPU. El entrenamiento será con CPU.")
    device = 'cpu'

# Configuración de Rutas
DATA_YAML = '../../datasets/01_autos/data.yaml'
PROJECT_DIR = '../../models/01_autos' # Donde guardaremos los resultados

PyTorch Version: 2.9.1+cu126
CUDA Available: True
Hardware detectado: NVIDIA GeForce RTX 4050 Laptop GPU
   Memoria VRAM: 6.05 GB


In [2]:
# --- baseline ---
# Ruta donde están los datos originales (sin limpiar)
BASELINE_DATA_PATH = os.path.abspath('../../data/processed/01_autos_original')

# Crear diccionario de configuración
yaml_baseline = {
    'path': BASELINE_DATA_PATH, # Ruta Absoluta
    'train': 'train/images',    # Entrenamiento
    'val': 'val/images',        # Validación
    'test': 'test/images',      # Pruebas
    'names': { 0: 'car' }       # Clases
}

# Guardar el archivo data_baseline.yaml
yaml_path_baseline = os.path.join(BASELINE_DATA_PATH, 'data_baseline.yaml')

with open(yaml_path_baseline, 'w') as f:
    yaml.dump(yaml_baseline, f)

print(f"Configuración Baseline creada en: {yaml_path_baseline}")

Configuración Baseline creada en: /home/roberto/proyecto/data/processed/01_autos_original/data_baseline.yaml


In [ ]:
# --- Entrenamiento baseline ---
print("Entrenamiento baseline (Datos Originales)...")

# Cargar modelo base
model_baseline = YOLO('yolov8n.pt') 

from datetime import datetime
run_name = f"{datetime.now().strftime('%Y%m%d')}_v8n_baseline_dataset"

# Entrenar
results_base = model_baseline.train(
    data=yaml_path_baseline,  # Apuntamos al YAML creado en la celda anterior
    project=PROJECT_DIR,      # ../../models/01_autos
    name=run_name,   # Nombre del experimento

    # Hiperparámetros Clave
    epochs=30,             # Máximo de pasadas (100)
    patience=20,            # Si no mejora en 20 épocas, parar (Early Stopping)
    batch=-1,               # AutoBatch (Usa el 90% de tu VRAM automáticamente)
    imgsz=640,              # Resolución de entrada estándar
    
    # Configuración de Hardware
    device=device,
    workers=6,              # Hilos de CPU para cargar datos

    # Gestión de Archivos    
    exist_ok=False,         # No sobrescribir si existe
    pretrained=True,        # Usar pesos de COCO (Transfer Learning)
    optimizer='auto',       # SGD o AdamW (Auto lo elige bien)
    verbose=True,
)

print("Entrenamiento Baseline finalizado.")

Entrenamiento baseline (Datos Originales)...
New https://pypi.org/project/ultralytics/8.3.232 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.229 🚀 Python-3.11.14 torch-2.9.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 11872MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/roberto/proyecto/data/processed/01_autos_original/data_baseline.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, 

In [ ]:
print("Iniciando validación Baseline en el set de Prueba...")

run_name = f"{datetime.now().strftime('%Y%m%d')}_v8n_baseline_dataset"

# Cargar el MEJOR modelo resultante del entrenamiento anterior
best_model_path = os.path.join(PROJECT_DIR, run_name, 'weights', 'best.pt')
best_model = YOLO(best_model_path)

# Ejecutar validación en split='test'
metrics = best_model.val(split='test', project=PROJECT_DIR, name=f"{run_name}_eval")

print(f"\nResultados Finales en Test del dataset Baseline:")
print(f"   mAP@50:    {metrics.box.map50:.4f} (Precisión holgada)")
print(f"   mAP@50-95: {metrics.box.map:.4f}  (Precisión estricta )")

Iniciando validación Baseline en el set de Prueba...
Ultralytics 8.3.229 🚀 Python-3.11.14 torch-2.9.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 11872MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 246.1±126.4 MB/s, size: 79.2 KB)
val: Scanning /home/roberto/proyecto/data/processed/01_autos_original/test/labels... 1618 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1618/1618 3.7Kit/s 0.4s0.0s
val: New cache created: /home/roberto/proyecto/data/processed/01_autos_original/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 102/102 23.2it/s 4.4s0.0s
                   all       1618       1618      0.999      0.999      0.995      0.958
Speed: 0.4ms preprocess, 0.8ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /home/roberto/proyecto/models/01_autos/20251126_v8n_baseline_dataset_eval

Resultados Finales

In [3]:
# 1. Cargar modelo base pre-entrenado (Transfer Learning)
# Opciones: yolov8n.pt (rápido), yolov8s.pt, yolov8m.pt (balanceado), yolov8l.pt (preciso)
model_variant = 'yolov8n.pt' 
model = YOLO(model_variant)

print(f"Modelo cargado: {model_variant}")

# 2. Configurar nombre del experimento (Para tu tesis: FECHA_MODELO_DATASET)
from datetime import datetime
run_name = f"{datetime.now().strftime('%Y%m%d')}_v8m_clean_dataset"

print(f"Los resultados se guardarán en: {PROJECT_DIR}/{run_name}")

Modelo cargado: yolov8n.pt
Los resultados se guardarán en: ../../models/01_autos/20251126_v8m_clean_dataset


In [26]:
# --- INICIAR ENTRENAMIENTO ---
results = model.train(
    data=DATA_YAML,
    
    # Hiperparámetros Clave
    epochs=30,             # Máximo de pasadas (100)
    patience=20,            # Si no mejora en 20 épocas, parar (Early Stopping)
    batch=-1,               # AutoBatch (Usa el 90% de tu VRAM automáticamente)
    imgsz=640,              # Resolución de entrada estándar
    
    # Configuración de Hardware
    device=device,
    workers=6,              # Hilos de CPU para cargar datos
    
    # Gestión de Archivos
    project=PROJECT_DIR,    # Carpeta raíz
    name=run_name,          # Subcarpeta del experimento
    exist_ok=False,         # No sobrescribir si existe
    pretrained=True,        # Usar pesos de COCO (Transfer Learning)
    optimizer='auto',       # SGD o AdamW (Auto lo elige bien)
    verbose=True,
    
    # Aumentación (Opcional: Desactiva HSV si prefieres ser estricto con B&W)
    hsv_h=0.0, hsv_s=0.0, hsv_v=0.4 
)

New https://pypi.org/project/ultralytics/8.3.232 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.229 🚀 Python-3.11.14 torch-2.9.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 11872MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../../datasets/01_autos/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=20251126_v8m_clean

In [16]:
print("Iniciando validación con el resplit en el set de Prueba...")

run_name = f"{datetime.now().strftime('%Y%m%d')}_v8m_clean_dataset"

# Cargar el MEJOR modelo resultante del entrenamiento anterior
best_model_path = os.path.join(PROJECT_DIR, run_name, 'weights', 'best.pt')
best_model = YOLO(best_model_path)

# Ejecutar validación en split='test'
metrics = best_model.val(split='test', project=PROJECT_DIR, name=f"{run_name}_eval")

print(f"\nResultados Finales en Test Set:")
print(f"   mAP@50:    {metrics.box.map50:.4f} (Precisión holgada)")
print(f"   mAP@50-95: {metrics.box.map:.4f}  (Precisión estricta )")

Iniciando validación con el resplit en el set de Prueba...
Ultralytics 8.3.229 🚀 Python-3.11.14 torch-2.9.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 11872MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1375.9±890.0 MB/s, size: 21.9 KB)
val: Scanning /home/roberto/proyecto/datasets/01_autos/test/labels.cache... 1615 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1615/1615 4.2Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 101/101 23.2it/s 4.4s.1s
                   all       1615       1615      0.998      0.999      0.995      0.954
Speed: 0.4ms preprocess, 0.7ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /home/roberto/proyecto/models/01_autos/20251126_v8m_clean_dataset_eval2

Resultados Finales en Test Set:
   mAP@50:    0.9948 (Precisión holgada)
   mAP@50-95: 0.9541  (Precisión estricta )


In [4]:
# 1. Cargar modelo desde cero con únicamente el dataset
# Opciones: yolov8n.pt (rápido), yolov8s.pt, yolov8m.pt (balanceado), yolov8l.pt (preciso)
model_variant = 'yolov8n.pt' 
model_0 = YOLO(model_variant)

print(f"Modelo cargado: {model_variant}")

# 2. Configurar nombre del experimento (Para tu tesis: FECHA_MODELO_DATASET)
from datetime import datetime
run_name = f"{datetime.now().strftime('%Y%m%d')}_v8m_clean_dataset"

print(f"Los resultados se guardarán en: {PROJECT_DIR}/{run_name}")

Modelo cargado: yolov8n.pt
Los resultados se guardarán en: ../../models/01_autos/20251126_v8m_clean_dataset


In [ ]:
# --- INICIAR ENTRENAMIENTO ---
results = model_0.train(
    data=DATA_YAML,
    
    # Hiperparámetros Clave
    epochs=30,             # Máximo de pasadas (100)
    patience=20,            # Si no mejora en 20 épocas, parar (Early Stopping)
    batch=-1,               # AutoBatch (Usa el 90% de tu VRAM automáticamente)
    imgsz=640,              # Resolución de entrada estándar
    
    # Configuración de Hardware
    device=device,
    workers=6,              # Hilos de CPU para cargar datos
    
    # Gestión de Archivos
    project=PROJECT_DIR,    # Carpeta raíz
    name=run_name,          # Subcarpeta del experimento
    exist_ok=False,         # No sobrescribir si existe
    pretrained=False,        # SIN UTILIZAR PESOS
    optimizer='auto',       # SGD o AdamW (Auto lo elige bien)
    verbose=True,
    
    # Aumentación (Opcional: Desactiva HSV si prefieres ser estricto con B&W)
    hsv_h=0.0, hsv_s=0.0, hsv_v=0.4 
)

New https://pypi.org/project/ultralytics/8.3.232 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.229 🚀 Python-3.11.14 torch-2.9.1+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 11872MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../../datasets/01_autos/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=20251126_v8m_clean

In [6]:
# Cargar métricas finales de los archivos guardados PENDIENTE
import pandas as pd

# Rutas de los CSV de resultados
base_csv = os.path.join(PROJECT_DIR, '20251126_v8n_baseline_dataset', 'results.csv')
ltrans_csv = os.path.join(PROJECT_DIR, '20251126_v8m_clean_dataset', 'results.csv')
propio_csv = os.path.join(PROJECT_DIR, '20251126_v8m_clean_dataset2', 'results.csv')

def get_best_map(csv_path):
    try:
        df = pd.read_csv(csv_path)
        # Limpiar nombres de columnas (YOLO pone espacios)
        df.columns = [c.strip() for c in df.columns]
        # Devolver el mejor mAP50-95 alcanzado
        return df['metrics/mAP50-95(B)'].max()
    except:
        return 0.0

map_base = get_best_map(base_csv)
map_ltrans = get_best_map(ltrans_csv)
map_propio = get_best_map(propio_csv)

print(f"\nRESULTADOS COMPARATIVOS (mAP 50-95):")
print(f"   - Baseline (Original): {map_base:.4f}")
print(f"   - Propuesto (Limpio):  {map_ltrans:.4f}")
print(f"   - Propuesto (Propio):  {map_propio:.4f}")

delta = ((map_ltrans - map_base) / map_base) * 100
print(f"Mejora relativa:    {delta:+.2f}%")


RESULTADOS COMPARATIVOS (mAP 50-95):
   - Baseline (Original): 0.9580
   - Propuesto (Limpio):  0.9545
   - Propuesto (Propio):  0.9545
Mejora relativa:    -0.36%


In [18]:
# Obtener imágenes del set de test para predecir
test_images = glob.glob('../../datasets/01_autos/test/images/*.jpg')[:4] # Tomamos 4 ejemplos

for img_path in test_images:
    # Inferencia
    results = best_model.predict(img_path, conf=0.5)
    
    # Mostrar
    for r in results:
        im_array = r.plot()  # r.plot() dibuja las cajas en la imagen numpy
        plt.figure(figsize=(8,8))
        plt.imshow(cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f"Detección: {os.path.basename(img_path)}")
        plt.show()


image 1/1 /home/roberto/proyecto/notebooks/01_autos/../../datasets/01_autos/test/images/013969.jpg: 480x640 1 car, 30.8ms
Speed: 1.6ms preprocess, 30.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 800x800 with 1 Axes>


image 1/1 /home/roberto/proyecto/notebooks/01_autos/../../datasets/01_autos/test/images/007246.jpg: 480x640 1 car, 4.1ms
Speed: 0.8ms preprocess, 4.1ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 800x800 with 1 Axes>


image 1/1 /home/roberto/proyecto/notebooks/01_autos/../../datasets/01_autos/test/images/010317.jpg: 480x640 1 car, 4.9ms
Speed: 1.5ms preprocess, 4.9ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 800x800 with 1 Axes>


image 1/1 /home/roberto/proyecto/notebooks/01_autos/../../datasets/01_autos/test/images/011789.jpg: 480x640 1 car, 4.3ms
Speed: 1.1ms preprocess, 4.3ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)


<Figure size 800x800 with 1 Axes>

In [19]:
import shutil

PRODUCTION_PATH = '../../production_weights'
os.makedirs(PRODUCTION_PATH, exist_ok=True)

# Copiar y renombrar
target_path = os.path.join(PRODUCTION_PATH, '01_autos_best.pt')
shutil.copy(best_model_path, target_path)

print(f"Modelo exportado exitosamente a: {target_path}")
print("   Este archivo está listo para ser usado por el Pipeline de Placas.")

Modelo exportado exitosamente a: ../../production_weights/01_autos_best.pt
   Este archivo está listo para ser usado por el Pipeline de Placas.
